In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
from elasticsearch import Elasticsearch
# create client for elastic search service running locally
es_client = Elasticsearch('http://localhost:9200') 
# es_client.info()

In [4]:
es_client.indices.delete(index='course-questions', ignore=[400, 404])

/tmp/ipykernel_3799/718326979.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.delete(index='course-questions', ignore=[400, 404])


ObjectApiResponse({'error': {'root_cause': [{'type': 'index_not_found_exception', 'reason': 'no such index [course-questions]', 'resource.type': 'index_or_alias', 'resource.id': 'course-questions', 'index_uuid': '_na_', 'index': 'course-questions'}], 'type': 'index_not_found_exception', 'reason': 'no such index [course-questions]', 'resource.type': 'index_or_alias', 'resource.id': 'course-questions', 'index_uuid': '_na_', 'index': 'course-questions'}, 'status': 404})

In [5]:
# create an index to use in elastic search
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
# git it a name
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
index_name = "course-questions"

In [6]:
from tqdm.auto import tqdm

In [7]:
# index all documents
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [23]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp" #"data-engineering-zoomcamp"
                        # "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    # print(response)
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [9]:
q3 = "How do I execute a command in a running docker container?"

In [24]:
elastic_search(q3)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [71]:
from openai import OpenAI
import os
client = OpenAI()
# os.environ['OPENAI_API_KEY']

In [14]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [25]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context +  context_template.format(question=doc['question'], text=doc['text']) +"\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [26]:
prompt = build_prompt(q3, elastic_search(q3))
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [27]:
len(prompt)

2714

In [66]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [36]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [68]:
rag(q3)

'To execute a command in a running Docker container, you can use the following steps:\n\n1. First, get the container ID of the running container by listing all running containers:\n   ```sh\n   docker ps\n   ```\n\n2. Once you have the container ID, use the `docker exec` command to execute a command inside the specific container. For example, to start a bash shell in the running container:\n   ```sh\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual ID of your running container.\n\nThis set of commands will allow you to execute commands inside a running Docker container.'

In [28]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [29]:
len(encoding.encode(prompt))

621